In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from pprint import pprint

# Credit Risk Score Prediction on Aave Protocol v2

*Disclaimer: this effort is made with limited knowledge on aave governance, and hence some assumptions are addressed as necessary*

In [9]:
# Source: https://towardsdatascience.com/how-to-develop-a-credit-risk-model-and-scorecard-91335fc01f03
# https://github.com/finlytics-hub/credit_risk_model
def run_query(query):

    # endpoint for making the request
    # https://api.thegraph.com/subgraphs/name/aave/protocol
    # https://api.thegraph.com/subgraphs/name/aave/protocol-multy-raw
    request = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                            '',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

In [83]:
users_query = """
{
  users(where: {
    id_not_in: ["0x0000000000000000000000000000000000000001",
                "0x000000000000000000000000000000000000dead"]
  }){
    id
    unclaimedRewards
    lifetimeRewards
    depositHistory(orderBy: timestamp, orderDirection: desc,) {
      id
      amount
      timestamp
      reserve {
        symbol
        name
        price{
          priceInEth
        }
        liquidityRate
        variableBorrowRate
        stableBorrowRate
      }
    }
    borrowHistory(orderBy: timestamp, orderDirection: desc) {
      id
      amount
      timestamp
      reserve {
        symbol
        name
        price{
          priceInEth
        }
        liquidityRate
        variableBorrowRate
        stableBorrowRate
      }
      borrowRate
      borrowRateMode
      stableTokenDebt
      variableTokenDebt
    }
    repayHistory{
      id
      amount
      timestamp
      reserve{
        symbol
        name
        price{
          priceInEth
        }
        liquidityRate
        variableBorrowRate
        stableBorrowRate
      }
    }
    redeemUnderlyingHistory{
      id
      amount
      timestamp
      reserve{
        symbol
        name
        price{
          priceInEth
        }
        liquidityRate
        variableBorrowRate
        stableBorrowRate
      }
    }
    incentivizedActions{
      id
      amount
    }
  }
}
"""


In [98]:
users_result = run_query(users_query)
#pprint(user_result)
#users_result['data']['users']

In [79]:
def extract_deposit_history(user_id, deposit=None):
    """
    Extract deposit history from a given user
    """
        
    deposit_dict = dict(
        user_id = user_id,
        deposit_reserve_name = '',
        deposit_reserve_symbol = '',
        deposit_reserve_priceInEth = 0,
        deposit_reserve_liquidityRate = 0,
        deposit_reserve_variableBorrowRate = 0,
        deposit_reserve_stableBorrowRate = 0,
        deposit_amount = 0,
        deposit_timestamp_unix = None,
        deposit_timestamp = None
    )
    
    if deposit:
        deposit_dict['deposit_reserve_name'] = deposit['reserve']['name']
        deposit_dict['deposit_reserve_symbol'] = deposit['reserve']['symbol']
        deposit_dict['deposit_reserve_priceInEth'] = int(deposit['reserve']['price']['priceInEth'])
        deposit_dict['deposit_reserve_liquidityRate'] = int(deposit['reserve']['liquidityRate'])
        deposit_dict['deposit_reserve_variableBorrowRate'] = int(deposit['reserve']['variableBorrowRate'])
        deposit_dict['deposit_reserve_stableBorrowRate'] = int(deposit['reserve']['stableBorrowRate'])
        deposit_dict['deposit_amount'] = int(deposit['amount'])
        deposit_dict['deposit_timestamp_unix'] = deposit['timestamp']
        deposit_dict['deposit_timestamp'] = pd.to_datetime(deposit['timestamp'], 
                                                           unit='s', errors='ignore')
        
    return deposit_dict

def extract_borrow_history(user_id, borrow=None):
    """
    Extract borrow history from a given user
    """
    borrow_dict = dict(
        user_id = user_id,
        borrow_reserve_name = '',
        borrow_reserve_symbol = '',
        borrow_reserve_priceInEth = 0,
        borrow_reserve_liquidityRate = 0,
        borrow_reserve_variableBorrowRate = 0,
        borrow_reserve_stableBorrowRate = 0,
        borrow_amount = 0,
        borrow_timestamp_unix = None,
        borrow_timestamp = None,
        borrow_rate = 0,
        borrow_rate_mode = '',
        borrow_stableTokenDebt = 0,
        borrow_variableTokenDebt = 0
        #borrow_accrued_interest = 0
    )
    
    if borrow:
        borrow_dict['borrow_reserve_name'] = borrow['reserve']['name']
        borrow_dict['borrow_reserve_symbol'] = borrow['reserve']['symbol']
        borrow_dict['borrow_reserve_priceInEth'] = int(borrow['reserve']['price']['priceInEth'])
        borrow_dict['borrow_reserve_liquidityRate'] = int(borrow['reserve']['liquidityRate'])
        borrow_dict['borrow_reserve_variableBorrowRate'] = int(borrow['reserve']['variableBorrowRate'])
        borrow_dict['borrow_reserve_stableBorrowRate'] = int(borrow['reserve']['stableBorrowRate'])
        borrow_dict['borrow_amount'] = int(borrow['amount'])
        borrow_dict['borrow_timestamp_unix'] = borrow['timestamp']
        borrow_dict['borrow_timestamp'] = pd.to_datetime(borrow['timestamp'],
                                                         unit='s', errors='ignore')
        borrow_dict['borrow_rate'] = int(borrow['borrowRate'])
        borrow_dict['borrow_rate_mode'] = borrow['borrowRateMode']
        borrow_dict['borrow_stableTokenDebt'] = borrow['stableTokenDebt']
        borrow_dict['borrow_variableTokenDebt'] = borrow['variableTokenDebt']
        #borrow_dict['borrow_accrued_interest'] = int(borrow['accruedBorrowInterest'])
    
    return borrow_dict

def extract_repay_history(user_id, repay=None):
    """
    Extract repay history from a given user
    """
    repay_dict = dict(
        user_id = user_id,
        repay_reserve_name = '',
        repay_reserve_symbol = '',
        repay_reserve_priceInEth = 0,
        repay_reserve_liquidityRate = 0,
        repay_reserve_variableBorrowRate = 0,
        repay_reserve_stableBorrowRate = 0,
        repay_amount = 0,
        repay_timestamp_unix = None,
        repay_timestamp = None
    )
    
    if repay:
        repay_dict['repay_reserve_name'] = repay['reserve']['name']
        repay_dict['repay_reserve_symbol'] = repay['reserve']['symbol']
        repay_dict['repay_reserve_priceInEth'] = int(repay['reserve']['price']['priceInEth'])
        repay_dict['repay_reserve_liquidityRate'] = int(repay['reserve']['liquidityRate'])
        repay_dict['repay_reserve_variableBorrowRate'] = int(repay['reserve']['variableBorrowRate'])
        repay_dict['repay_reserve_stableBorrowRate'] = int(repay['reserve']['stableBorrowRate'])
        repay_dict['repay_amount'] = int(repay['amount'])
        repay_dict['repay_timestamp_unix'] = repay['timestamp']
        repay_dict['repay_timestamp'] = pd.to_datetime(repay['timestamp'],
                                                       unit='s', errors='ignore')

    return repay_dict


def extract_redeemUnderlying_history(user_id, redeem=None):
    
    redeem_dict = dict(
        user_id = user_id,
        redeem_reserve_name = '',
        redeem_reserve_symbol = '',
        redeem_reserve_priceInEth = 0,
        redeem_reserve_liquidityRate = 0,
        redeem_reserve_variableBorrowRate = 0,
        redeem_reserve_stableBorrowRate = 0,
        redeem_amount = 0,
        redeem_timestamp_unix = None,
        redeem_timestamp = None
    )
    
    if redeem:
        redeem_dict['redeem_reserve_name'] = redeem['reserve']['name']
        redeem_dict['redeem_reserve_symbol'] = redeem['reserve']['symbol']
        redeem_dict['redeem_reserve_priceInEth'] = int(redeem['reserve']['price']['priceInEth'])
        redeem_dict['redeem_reserve_liquidityRate'] = int(redeem['reserve']['liquidityRate'])
        redeem_dict['redeem_reserve_variableBorrowRate'] = int(redeem['reserve']['variableBorrowRate'])
        redeem_dict['redeem_reserve_stableBorrowRate'] = int(redeem['reserve']['stableBorrowRate'])
        redeem_dict['redeem_amount'] = int(redeem['amount'])
        redeem_dict['redeem_timestamp_unix'] = int(redeem['timestamp'])
        redeem_dict['redeem_timestamp'] = pd.to_datetime(redeem['timestamp'],
                                                         unit='s', errors='ignore')

    return redeem_dict
    



In [94]:
users, deposits, borrows, repays, redeems = [], [], [], [], []
for user in users_result['data']['users']:
    #print(user, '\n')
    
    user_id = user['id']
        
    users.append(dict(
        user_id = user_id,
        user_unclaimedRewards = user['unclaimedRewards'],
        user_lifetimeRewards = user['lifetimeRewards'],
        user_incentivizedActions = int(user['incentivizedActions'][0]['amount']) \
            if user['incentivizedActions'] else 0
    ))
    
    # extract depositHistory
    if user['depositHistory']:
        for deposit in user['depositHistory']:
            deposit_dict = extract_deposit_history(user_id, deposit)
            deposits.append(deposit_dict)
    else:
        deposit_dict = extract_deposit_history(user_id)
        deposits.append(deposit_dict)
    
    # extract borrowHistory
    if user['borrowHistory']:
        for borrow in user['borrowHistory']:
            borrow_dict = extract_borrow_history(user_id, borrow)
            borrows.append(borrow_dict)
    else:
        borrow_dict = extract_borrow_history(user_id)
        borrows.append(borrow_dict)
   
    # extract repayHistory
    if user['repayHistory']:
        for repay in  user['repayHistory']:
            repay_dict = extract_repay_history(user_id, repay)
            repays.append(repay_dict)
    else:
        repay_dict = extract_repay_history(user_id)
        repays.append(repay_dict)
        
    # extract redeemUnderlyingHistory
    if user['redeemUnderlyingHistory']:
        for redeem in user['redeemUnderlyingHistory']:
            redeem_dict = extract_redeemUnderlying_history(user_id, redeem)
            redeems.append(redeem_dict)
    else:
        redeem_dict = extract_redeemUnderlying_history(user_id)
        redeems.append(redeem_dict)
            

In [96]:
users_df = pd.DataFrame(users)
users_df.head(50)

,user_id,user_unclaimedRewards,user_lifetimeRewards,user_incentivizedActions
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,46,46,4
1,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,7217726631361778,7217726631361778,2964438606538810
2,0x00000000005dbcb0d0513fcda746382fe8a53468,0,0,0
3,0x00000000009a41862f3b2b0c688b7c0d1940511e,0,0,0
4,0x0000000000d41c96294ccdac8612bdfe29c641af,0,0,0
5,0x000000000a38444e0a6e37d3b630d7e855a7cb13,1172838535267861,1172838535267861,3286256212453
6,0x000000003ce0cf2c037493b1dc087204bd7f713e,4594031588,4594031588,4594031588
7,0x0000000094acb89a43eac2fbb3a07973efc2435c,0,0,0
8,0x0000000484f2217f1a64eb6d24b5cee446faeae5,0,0,0
9,0x0000006daea1723962647b7e189d311d757fb793,12000794475704540712,12000794475704540712,12708916691465


In [97]:
users_df[users_df.user_unclaimedRewards != users_df.user_lifetimeRewards]

,user_id,user_unclaimedRewards,user_lifetimeRewards,user_incentivizedActions
40,0x00137878c7c179cad13505c18c86fd84623f04c7,0,1643451823147600233,514903473
42,0x0013eb108d5168bf2285f0f54172394f7c17d17d,0,7243024470815893431,7243024470815893431
43,0x0014a4f88a92d3924a3f553dc51c290ce06ae125,45197643421884619,13346803142920585986,3866509416886132830
58,0x002ad2406d6d05e0c58e39af6eb506f762e9f7bb,0,2954459110650692796,378448660935842508
64,0x0032311e56bc34efde20b408bb99d986eea1863a,88526724098484314,477083392433139171,13889109675855785
80,0x00405dc6b34d782df3feb0f0ec308fc27d4bd6bb,0,3802339088532774779,705246126826882783
90,0x004c350cd1ab72ea28cd6c47935b5bb31f64a928,0,1305510934083713815,694339479806045139
92,0x004f137c71b3003c7915bd6aa3a4f57e334a738e,0,290784981422828450,79740662718619354


In [72]:
deposits_df = pd.DataFrame(deposits)
deposits_df.head(50)

,user_id,deposit_reserve_name,deposit_reserve_symbol,deposit_reserve_priceInEth,deposit_reserve_liquidityRate,deposit_reserve_variableBorrowRate,deposit_reserve_stableBorrowRate,deposit_amount,deposit_timestamp_unix,deposit_timestamp
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,127090210000000016,1.626330e+09,2021-07-15 06:18:08
1,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,115695250000000000,1.626201e+09,2021-07-13 18:30:44
2,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,115019330000000000,1.626196e+09,2021-07-13 17:00:22
3,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,132166879999999984,1.626074e+09,2021-07-12 07:16:40
4,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,114580540000000000,1.625835e+09,2021-07-09 12:50:26
5,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,147193320000000000,1.621584e+09,2021-05-21 07:59:55
6,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,262418969999999968,1.619550e+09,2021-04-27 19:05:48
7,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,257442980000000000,1.619501e+09,2021-04-27 05:25:17
8,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,272946919999999968,1.619500e+09,2021-04-27 05:00:03
9,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,209106470000000000,1.619480e+09,2021-04-26 23:36:16


In [73]:
borrows_df = pd.DataFrame(borrows)
borrows_df.head(50)

,user_id,borrow_reserve_name,borrow_reserve_symbol,borrow_reserve_priceInEth,borrow_reserve_liquidityRate,borrow_reserve_variableBorrowRate,borrow_reserve_stableBorrowRate,borrow_amount,borrow_timestamp_unix,borrow_timestamp,borrow_rate,borrow_rate_mode,borrow_stableTokenDebt,borrow_variableTokenDebt
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,,,0,0,0,0,0,NaN,NaT,0,,0,0
1,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,Synthetix Network Token,SNX,3935000000000000,48549344424322288981854689,121905074094503566072178006,0,5000000000000000000000,1.622545e+09,2021-06-01 11:00:43,127312792424753718439733675,Variable,0,4760354075421655119279
2,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,Republic Token,REN,174430000000000,1197028155730986160423701,15066428760840451542085527,21523469658343502202979325,100000000000000000000000,1.622303e+09,2021-05-29 15:35:30,8024003551450866705556542,Variable,0,98740619248157900299254
3,0x00000000005dbcb0d0513fcda746382fe8a53468,,,0,0,0,0,0,NaN,NaT,0,,0,0
4,0x00000000009a41862f3b2b0c688b7c0d1940511e,,,0,0,0,0,0,NaN,NaT,0,,0,0
5,0x0000000000d41c96294ccdac8612bdfe29c641af,,,0,0,0,0,0,NaN,NaT,0,,0,0
6,0x000000000a38444e0a6e37d3b630d7e855a7cb13,Ampleforth,AMPL,370000000000000,2610028615943508391089334,10428383975045199508986711,0,202229999383,1.627248e+09,2021-07-25 21:21:13,480000000000000000000000000,Variable,0,1231671341777
7,0x000000000a38444e0a6e37d3b630d7e855a7cb13,Ampleforth,AMPL,370000000000000,2610028615943508391089334,10428383975045199508986711,0,1030729413758,1.627229e+09,2021-07-25 16:01:24,480000000000000000000000000,Variable,0,1029701876086
8,0x000000003ce0cf2c037493b1dc087204bd7f713e,,,0,0,0,0,0,NaN,NaT,0,,0,0
9,0x0000000094acb89a43eac2fbb3a07973efc2435c,,,0,0,0,0,0,NaN,NaT,0,,0,0


In [74]:
no_deposits = set(deposits_df[deposits_df.deposit_amount == 0].user_id)
no_borrows = set(borrows_df[borrows_df.borrow_amount == 0].user_id)

print(len(no_deposits))
print(len(no_borrows))

len(no_deposits.intersection(no_borrows))

23
69


16

In [75]:
repays_df = pd.DataFrame(repays)
repays_df.head(50)

,user_id,repay_reserve_name,repay_reserve_symbol,repay_reserve_priceInEth,repay_reserve_liquidityRate,repay_reserve_variableBorrowRate,repay_reserve_stableBorrowRate,repay_amount,repay_timestamp_unix,repay_timestamp
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,,,0,0,0,0,0,NaN,NaT
1,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,Synthetix Network Token,SNX,3935000000000000,48549344424322288981854689,121905074094503566072178006,0,5000478484297793675639,1.622568e+09,2021-06-01 17:21:43
2,0x00000000000cd56832ce5dfbcbff02e7ec639bc9,Republic Token,REN,174430000000000,1197028155730986160423701,15066428760840451542085527,21523469658343502202979325,100000840317343090649398,1.622335e+09,2021-05-30 00:42:31
3,0x00000000005dbcb0d0513fcda746382fe8a53468,,,0,0,0,0,0,NaN,NaT
4,0x00000000009a41862f3b2b0c688b7c0d1940511e,,,0,0,0,0,0,NaN,NaT
5,0x0000000000d41c96294ccdac8612bdfe29c641af,,,0,0,0,0,0,NaN,NaT
6,0x000000000a38444e0a6e37d3b630d7e855a7cb13,Ampleforth,AMPL,370000000000000,2610028615943508391089334,10428383975045199508986711,0,1237311201614,1.627464e+09,2021-07-28 09:25:22
7,0x000000003ce0cf2c037493b1dc087204bd7f713e,,,0,0,0,0,0,NaN,NaT
8,0x0000000094acb89a43eac2fbb3a07973efc2435c,,,0,0,0,0,0,NaN,NaT
9,0x0000000484f2217f1a64eb6d24b5cee446faeae5,Wrapped Ether,WETH,1000000000000000000,237627369906352074450742,5524461687986718164801825,36905577109983397706002281,300002875869535989282,1.617127e+09,2021-03-30 17:55:39


In [82]:
redeems_df = pd.DataFrame(redeems)
redeems_df.head(50)

,user_id,redeem_reserve_name,redeem_reserve_symbol,redeem_reserve_priceInEth,redeem_reserve_liquidityRate,redeem_reserve_variableBorrowRate,redeem_reserve_stableBorrowRate,redeem_amount,redeem_timestamp_unix,redeem_timestamp
0,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,248413545665620448,1.619287e+09,2021-04-24 18:02:27
1,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,258171608256478919,1.620546e+09,2021-05-09 07:36:20
2,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,319815822440612672,1.621407e+09,2021-05-19 06:50:05
3,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Decentraland MANA,MANA,288041683600522,2488467086957333055285278,24276763042297268535444511,64681090060424669336349302,14829590341096803663872,1.620104e+09,2021-05-04 04:49:37
4,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,405518338003156435,1.617909e+09,2021-04-08 19:08:33
5,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,405044403648473472,1.618136e+09,2021-04-11 10:16:39
6,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,225027688739205941,1.619782e+09,2021-04-30 11:23:06
7,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,131738729861817776,1.626760e+09,2021-07-20 05:48:47
8,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,94420463230377568,1.624069e+09,2021-06-19 02:21:09
9,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,Wrapped Ether,WETH,1000000000000000000,237633987311149257994411,5524540890021408828519751,36905676112526761035649689,459740369875990448,1.617829e+09,2021-04-07 21:02:41
